In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
class Dataset:
    data = # numpy array n data points by m features
    targets = # numpy array (vector) length n
    
    def __init__(self, path_to_csv):
        data, targets = self.parse_csv(path_to_csv)
        
        self.data, self.targets = data, targets
        
    def parse_csv():
        
        self.encode(data)
        pass
    
    def encode(string):
        '''
        Converts a string into an binary array of features.
        
        Parameters
        ----------
        string: str
            Sequence of amino acids (all CAPS)
        
        Returns
        ----------
        features: ndarray
            Flat vector encoding each amino acid (N x 1)
        '''
        